In [ ]:
import numpy as np

print("Opening...")
with open("a1a.train") as f:
    train_raw = f.read()

with open("a1a.test") as f:
    test_raw = f.read()
print("done opening.")

def process_data(raw_data):
    train_lines = raw_data.splitlines()
    num_examples = len(train_lines)
    num_features = 123
    X = np.zeros((num_examples, num_features))
    Y = np.zeros((num_examples, 1))
    for i, line in enumerate(train_lines):
        tokens = line.split()
        #label = tokens[0]
        label = (int(tokens[0]) + 1) / 2  # Change label from {-1,1} to {0,1}
        Y[i] = label
        for token in tokens[1:]:
            index = int(token[:-2]) - 1
            X[i, index] = 1
    return X, Y

def normalize_data(Xtrain, Xtest):
    normalizer = max(np.max(np.linalg.norm(Xtrain, axis=1)),
                 np.max(np.linalg.norm(Xtest, axis=1)))
    Xtrain = Xtrain / normalizer
    Xtest = Xtest / normalizer
    return Xtrain, Xtest

print("Processing...")
Xtrain, Ytrain = process_data(train_raw)
Xtest, Ytest = process_data(test_raw)
print("done processing.")
print("Normalizing...")
Xtrain, Xtest = normalize_data(Xtrain, Xtest)
print("done normalizing.")

In [ ]:
from math import exp, sqrt
from scipy.special import erf
from scipy.optimize import root_scalar

def get_eps_AGM(sigma, GS, delta, min_eps=1e-6, max_eps=10, tol=1e-12):
    # Compute the epsilon corresponding to a Gaussian perturbation
    normalized_sigma = sigma / GS
    
    def Phi(t):
        return 0.5*(1.0 + erf(float(t)/sqrt(2.0)))
    def get_delta(s, e):
        return Phi(-e*s+1.0/(2*s)) - exp(e)*Phi(-e*s-1.0/(2*s))
    def f(x):
        return get_delta(normalized_sigma, x) - delta
    
    assert get_delta(normalized_sigma, min_eps) >= delta
    assert get_delta(normalized_sigma, max_eps) <= delta
    
    sol = root_scalar(f, bracket=[min_eps,max_eps], xtol=tol)
    assert sol.converged
    
    return sol.root

In [ ]:
get_eps_AGM(1,0.1,1e-6)

In [ ]:
import psgd

psgd.get_eps_AGM(0.1,1.3,1e-6,max_eps=500)

In [ ]:
# ProjSGDClassifier is an sklearn model that needs to be compiled locally
# See README in parent folder
from sklearn.linear_model import ProjSGDClassifier

def dp_proj_sgd(Xtrain, Xtest, reg_lambda=0.001, sigma=0.1, delta=1e-6, R=10):
    
    # Define the model
    clf = ProjSGDClassifier(loss="log", penalty="l2",
                            learning_rate="bolton",
                            alpha=reg_lambda,
                            radius=1.0/reg_lambda,
                            max_iter=10,
                            verbose=0,
                            fit_intercept=False)
    #print(clf.get_params())
    
    scores = []
    for r in range(R):       
        # Train the model
        clf.fit(Xtrain, Ytrain.ravel())
        # Privatize the model
        Z = sigma*np.random.standard_normal(size=clf.coef_.shape)
        clf.coef_ += Z
        # Evaluate the model accuracy
        score = clf.score(Xtest, Ytest)  
        scores.append(score)

    # Evaluate the model privacy
    # Compute the global sensitivity
    m = Xtrain.shape[0]
    GS = 4.0/(m*reg_lambda)
    epsilon = get_eps_AGM(sigma, GS, delta)
    
    return np.average(scores), epsilon

In [ ]:
dp_proj_sgd(Xtrain, Xtest, reg_lambda=0.01, sigma=0.1, delta=1e-6)